In [6]:
import glob
import rhksm4
import numpy as np
from matplotlib import pyplot as plt
import os
import sati
from IPython.display import clear_output
from scipy.stats import norm

def listdirs(rootdir):
    result = []
    for it in os.scandir(rootdir):
        if it.is_dir():
            result.append(it.path)
            #print(it.path)
            for inner_path in listdirs(it):
                result.append(inner_path)
    return result

def equalize(num, x1=1, x2=99):
    nums = num.reshape(num.shape[0]*num.shape[1])
    nums = np.sort(nums)
    x1 = nums[int(x1/100*len(nums))]
    x2 = nums[int(x2/100*len(nums))]
    num = num.clip(x1, x2)
    num = 255*(num-np.min(num))/(np.max(num)-np.min(num))
    return num

def sm4_microscopy_preprocess(dir, cmap, eq1, eq2):
    print('\n\nMicroscopy script:')
    path = os.getcwd()
    try:
        os.mkdir(path+'/sm4_microscopy_image')
    except OSError as error:
        pass
    if dir == 'current':
        names = glob.glob(path+'\\*.SM4')
    elif dir == 'inner':
        names = glob.glob(path+'\\*.SM4')
        for inner_path in listdirs(path):
            for name in glob.glob(inner_path+'\\*.SM4'):
                names.append(name)



    for name in names:
        title = name.split('\\')[-1].split('.')[0]
        print('\n\n'+title)
        try:
            f = rhksm4.load(name)
            for page in f:
                #print(page.label)
                if (page.label == ('Topography_Forward') or page.label == ('Aux_1__Forward')):
                    #print('\n\n'+title)
                    print(page.label)
                    num = np.array(page.data[:,::-1])    
                    num = (num-np.min(num))/(np.max(num)-np.min(num))
                    img1 = num*255
                    size1 = img1.shape[0]
                    size2 = img1.shape[1]
                    img1_p1 = img1[0:size1//2,0:size2//2] 
                    img1_p2 = img1[size1//2:size1,0:size2//2] 
                    img1_p3 = img1[0:size1//2,size2//2:size2] 
                    img1_p4 = img1[size1//2:size1,size2//2:size2]
                        
                    initrsp1 = sati.GuessInitRsp(img1_p1, n=1, threshold=254.5)
                    initrsp2 = sati.GuessInitRsp(img1_p2, n=1, threshold=254.5)
                    initrsp3 = sati.GuessInitRsp(img1_p3, n=1, threshold=254.5)
                    initrsp4 = sati.GuessInitRsp(img1_p4, n=1, threshold=254.5)
                    m1 = sati.Model(img1_p1, rsp=initrsp1.guess, dist=sati.distributions.Norm(), 
                                    poly=sati.planes.Poly())
                    m2 = sati.Model(img1_p2, rsp=initrsp2.guess, dist=sati.distributions.Norm(), 
                                    poly=sati.planes.Poly())
                    m3 = sati.Model(img1_p3, rsp=initrsp3.guess, dist=sati.distributions.Norm(), 
                                    poly=sati.planes.Poly())
                    m4 = sati.Model(img1_p4, rsp=initrsp4.guess, dist=sati.distributions.Norm(), 
                                    poly=sati.planes.Poly())
                    m1.optimize()
                    m2.optimize()
                    m3.optimize()
                    m4.optimize()

                    m1.subtracted= equalize(m1.subtracted,eq1,eq2)
                    m2.subtracted= equalize(m2.subtracted,eq1,eq2)
                    m3.subtracted= equalize(m3.subtracted,eq1,eq2)
                    m4.subtracted= equalize(m4.subtracted,eq1,eq2)

                    if cmap == 0:
                        fig,axs=plt.subplots(2,2,figsize=(7,7))
                        axs[0,0].imshow(m1.subtracted).set_cmap('afmhot')
                        axs[1,0].imshow(m2.subtracted).set_cmap('afmhot')
                        axs[0,1].imshow(m3.subtracted).set_cmap('afmhot')
                        axs[1,1].imshow(m4.subtracted).set_cmap('afmhot')
                    if cmap == 1:
                        fig,axs=plt.subplots(2,2,figsize=(7,7))
                        axs[0,0].imshow(m1.subtracted).set_cmap('hot')
                        axs[1,0].imshow(m2.subtracted).set_cmap('hot')
                        axs[0,1].imshow(m3.subtracted).set_cmap('hot')
                        axs[1,1].imshow(m4.subtracted).set_cmap('hot')    
                    plt.close(fig)
                    fig.savefig(path+'/sm4_microscopy_image/'+title+page.label+'.jpg')
        except:
            print('Error in '+title)
    #clear_output(wait=False)





def sm4_spectroscopy_preprocess(dir):
    print('\n\nSpectroscopy script:')
    
    path = os.getcwd()
    try:
        os.mkdir(path+'/sm4_spectroscopy_image')
    except OSError as error:
        pass
    if dir == 'current':
        names = glob.glob(path+'\\*.SM4')
    elif dir == 'inner':
        names = glob.glob(path+'\\*.SM4')
        for inner_path in listdirs(path):
            for name in glob.glob(inner_path+'\\*.SM4'):
                names.append(name)
    
    
    
    for name in names:
        title = name.split('\\')[-1].split('.')[0]
        print('\n\n'+title)
        try:
            f = rhksm4.load(name)
            for page in f:
                if (page.label == ('Aux_1_') or page.label == ('Current_')):
                    print(page.label)
                    num = np.array(page.data)
                    #print(num[0][::-1])
                    fig,axs=plt.subplots(1,figsize=(10,10))
                    axs.plot(num[0][::-1])
                    axs.grid()
                    #axs[0].set_xticklabels('xyZ',fontsize=15)
                    axs.plot(num[1][::-1])
                         

 
                    plt.close(fig)
                    fig.savefig(path+'/sm4_spectroscopy_image/'+title+'__'+page.label+'.jpg')
        except:
            print('Error in '+title)

if __name__ == "__main__":
    #sm4_microscopy_preprocess('current',1, 5, 95) 
    sm4_spectroscopy_preprocess('current')



Spectroscopy script:
['c:\\pythonprog\\sati\\20220904s0024.SM4']


20220904s0024
Aux_1_
Current_


In [48]:
f = rhksm4.load('c:\\pythonprog\\sati\\20220904s0024.SM4')
#print(f[0].label)
page=f[0]
print(page.data.shape)
#?page
print(page.attrs['RHK_Xoffset'])
print(page.attrs['RHK_Xscale'])
print(page.attrs['RHK_Xunits'])
print(page.attrs['RHK_Xsize'])
print(page.coords[1][1])


(2, 101)
0.39916992
-0.010070801
V
101
[ 0.39916992  0.38909912  0.37902832  0.36895752  0.35888672  0.34881592
  0.33874512  0.32867432  0.31860352  0.30853271  0.29846191  0.28839111
  0.27832031  0.26824951  0.25817871  0.24810791  0.23803711  0.22796631
  0.21789551  0.20782471  0.19775391  0.18768311  0.1776123   0.1675415
  0.1574707   0.1473999   0.1373291   0.1272583   0.1171875   0.1071167
  0.0970459   0.0869751   0.0769043   0.0668335   0.0567627   0.04669189
  0.03662109  0.02655029  0.01647949  0.00640869 -0.00366211 -0.01373291
 -0.02380371 -0.03387451 -0.04394531 -0.05401611 -0.06408691 -0.07415771
 -0.08422852 -0.09429932 -0.10437012 -0.11444092 -0.12451172 -0.13458252
 -0.14465332 -0.15472412 -0.16479492 -0.17486572 -0.18493652 -0.19500732
 -0.20507812 -0.21514893 -0.22521973 -0.23529053 -0.24536133 -0.25543213
 -0.26550293 -0.27557373 -0.28564453 -0.29571533 -0.30578613 -0.31585693
 -0.32592773 -0.33599854 -0.34606934 -0.35614014 -0.36621094 -0.37628174
 -0.38635254 -

In [51]:
f = rhksm4.load('c:\\pythonprog\\sati\\20220904s0024.SM4')
#print(f[0].label)
page=f[0]
print(page.data.shape)
#?page
print(page.attrs['RHK_Yoffset'])
print(page.attrs['RHK_Yscale'])
print(page.attrs['RHK_Yunits'])
print(page.attrs['RHK_Ysize'])


(2, 101)
0.0
1.0
 
2


In [20]:
f = rhksm4.load('c:\\pythonprog\\sati\\20220904s0024.SM4')
#print(f[0].label)
page=f[0]
#?page
for atr in page.attrs:
    #print(atr+'   '+page.attrs[atr])
    print(atr)


RHK_PRMdata
RHK_PageID
RHK_PageDataType
RHK_PageDataTypeName
RHK_PageSourceType
RHK_PageSourceTypeName
RHK_MinorVer
RHK_Signature
RHK_PageType
RHK_PageTypeName
RHK_DataSubSource
RHK_LineType
RHK_LineTypeName
RHK_Xcorner
RHK_Ycorner
RHK_Xsize
RHK_Ysize
RHK_ImageType
RHK_ImageTypeName
RHK_ScanType
RHK_ScanTypeName
RHK_GroupId
RHK_MinZvalue
RHK_MaxZvalue
RHK_Xscale
RHK_Yscale
RHK_Zscale
RHK_XYscale
RHK_Xoffset
RHK_Yoffset
RHK_Zoffset
RHK_Period
RHK_Bias
RHK_Current
RHK_Angle
RHK_GridXsize
RHK_GridYsize
RHK_Label
RHK_SystemText
RHK_SessionText
RHK_UserText
RHK_FileName
RHK_Date
RHK_Time
RHK_Xunits
RHK_Yunits
RHK_Zunits
RHK_Xlabel
RHK_Ylabel
RHK_StatusChannelText
RHK_CompletedLineCount
RHK_OverSamplingCount
RHK_SlicedVoltage
RHK_PLLProStatus
RHK_DateTime
RHK_SpecDrift_Filetime
RHK_SpecDrift_DriftOptionType
RHK_SpecDrift_DriftOptionTypeName
RHK_SpecDrift_Channel
RHK_SpecDrift_Time
RHK_SpecDrift_Xcoord
RHK_SpecDrift_Ycoord
RHK_SpecDrift_dX
RHK_SpecDrift_dY
RHK_SpecDrift_CumulativeX
RHK_SpecDr